In [133]:
import pandas as pd
import re
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
import pyLDAvis.gensim_models
import warnings
warnings.filterwarnings('ignore')

In [112]:
def convert_currency(amount):

    number = 1000 * float(re.search(r'\s(\d+\.\d+)', amount).group(1))

    currency = amount[:3]
    if currency != "EUR":
        if currency == "BOB":
            conv = 8.3725
        elif currency == "DKK":
            conv = 7.4374
        elif currency == "GEL":
            conv = 3.8279
        elif currency == "INR":
            conv = 88.6972
        elif currency == "JOD":
            conv = 0.8585
        elif currency == "KES":
            conv = 130.5909
        elif currency == "LKR":
            conv = 239.5998
        elif currency == "NGN":
            conv = 498.8897
        elif currency == "NPR":
            conv = 142.5807
        elif currency == "TRY":
            conv = 10.1572
        elif currency == "USD":
            conv = 1.2108
        elif currency == "ZAR":
            conv = 16.6102
        else:
            print("warning, non-defined currency in dataset")
            conv = 1

        number = number / conv

    return number

def check_search_term(description, search_terms):
    for term in search_terms:
        if term in description:
            return 1
    return 0

In [113]:
#Reading the data
folder = Path("/Users/nielskreuk/Dropbox/DataScience/Exercises/NYCDSA/ScrapingProject/fmo/scrapeddata")
file_to_open = folder / "fmo.csv"
cols = pd.read_csv(file_to_open, sep=";", nrows=1).columns
df = pd.read_csv(file_to_open, sep=";", usecols=cols[:6]) #Leave out couple of empty columns at end of file

In [114]:
df.columns

Index(['amount', 'country', 'date', 'description', 'industry', 'project_name'], dtype='object')

In [115]:
#add a column with converted amounts in EUR as numbers
df["EUR_amounts"] = [convert_currency(amount) for amount in df["amount"]]

In [116]:
# Preprocessing

# Note: Text already made lowercase during scraping
# Remove numbers
df['description'] = df['description'].apply(lambda x: re.sub('[0-9]+', ' ', x))
# Remove links
df['description'] = df['description'].apply(lambda x: re.sub('http\S*', '', x))
# Remove double spaces
df['description'] = df['description'].apply(lambda x: re.sub('\s+', ' ', x))



In [117]:
stemmer = PorterStemmer()
df["description"] = [stemmer.stem(word) for word in df["description"]]

In [118]:
count_vec = CountVectorizer(min_df=5)
count = count_vec.fit_transform(df["description"])

In [119]:
count.shape

(954, 2661)

In [120]:
print(count_vec.get_feature_names()[:100])

['abidjan', 'ability', 'able', 'about', 'above', 'abroad', 'abundant', 'accelerate', 'acceptable', 'access', 'accessibility', 'accessible', 'accessing', 'accion', 'accordance', 'according', 'account', 'accounts', 'achieve', 'achieved', 'achieving', 'acquire', 'acquired', 'acquiring', 'acquisition', 'across', 'act', 'acted', 'acting', 'action', 'actions', 'active', 'actively', 'activities', 'activity', 'acts', 'adb', 'add', 'added', 'adding', 'addition', 'additional', 'additionality', 'additionally', 'address', 'addressed', 'addresses', 'addressing', 'adds', 'adequate', 'adequately', 'adhere', 'adherence', 'adjacent', 'adopted', 'advance', 'advanced', 'advantage', 'advantages', 'adverse', 'advice', 'advisor', 'advisors', 'advisory', 'aef', 'afc', 'affairs', 'affected', 'affordability', 'affordable', 'africa', 'african', 'africinvest', 'after', 'ag', 'against', 'agencies', 'agency', 'agenda', 'agent', 'aggregate', 'agnostic', 'agreed', 'agreement', 'agreements', 'agri', 'agribusiness', '

In [121]:
tf_idf = TfidfVectorizer(min_df = 5, max_df = 0.75, stop_words = "english")
count_tf = tf_idf.fit_transform(df["description"])
count_tf.shape

(954, 2451)

In [122]:
print(tf_idf.get_feature_names()[::100])

['abidjan', 'allowing', 'begin', 'categories', 'complements', 'corridor', 'development', 'eligible', 'expert', 'form', 'growth', 'inception', 'issue', 'lines', 'meter', 'norfund', 'participation', 'practices', 'psc', 'replacing', 'sector', 'source', 'supplies', 'transportation', 'volume']


In [123]:
dictionary = gensim.corpora.Dictionary(df["description"].map(lambda x: x.split(' ')))

In [124]:
bow_corpus = [dictionary.doc2bow(wrd) for wrd in df["description"].map(lambda x: x.split(' '))]

In [125]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=6, id2word=dictionary, passes=2)
lda_model.save('lda.model')

In [126]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} Word: {}\n'.format(idx, topic))

Topic: 0 Word: 0.046*"the" + 0.032*"in" + 0.030*"and" + 0.019*"a" + 0.017*"to" + 0.017*"of" + 0.015*"is" + 0.010*"for" + 0.008*"with" + 0.008*"fund"

Topic: 1 Word: 0.052*"the" + 0.035*"and" + 0.034*"in" + 0.025*"to" + 0.024*"of" + 0.020*"a" + 0.017*"is" + 0.011*"will" + 0.008*"for" + 0.007*"by"

Topic: 2 Word: 0.038*"the" + 0.031*"in" + 0.024*"and" + 0.020*"of" + 0.020*"a" + 0.017*"to" + 0.015*"is" + 0.007*"with" + 0.006*"by" + 0.006*"will"

Topic: 3 Word: 0.067*"the" + 0.039*"and" + 0.033*"of" + 0.026*"to" + 0.024*"in" + 0.019*"is" + 0.017*"a" + 0.009*"with" + 0.009*"will" + 0.008*"project"

Topic: 4 Word: 0.048*"the" + 0.047*"and" + 0.035*"to" + 0.030*"in" + 0.028*"of" + 0.021*"a" + 0.018*"is" + 0.012*"with" + 0.011*"will" + 0.009*"for"

Topic: 5 Word: 0.039*"the" + 0.035*"and" + 0.032*"in" + 0.026*"to" + 0.024*"a" + 0.023*"of" + 0.022*"is" + 0.011*"with" + 0.010*"will" + 0.007*"fund"



In [127]:
tfidf = gensim.models.TfidfModel(bow_corpus)
tfidf_corpus = tfidf[bow_corpus]

In [128]:
lda_model_tfidf = gensim.models.LdaMulticore(tfidf_corpus, num_topics=6, id2word=dictionary, passes=2)
lda_model_tfidf.save('lda_tfidf.model')

In [129]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.001*"fund" + 0.001*"energy" + 0.001*"project" + 0.001*"water" + 0.001*"ps" + 0.001*"wind" + 0.001*"sme" + 0.001*"usd" + 0.001*"power" + 0.001*"company"
Topic: 1 Word: 0.001*"energy" + 0.001*"fund" + 0.001*"bank" + 0.001*"project" + 0.001*"ps" + 0.001*"solar" + 0.001*"power" + 0.001*"financial" + 0.001*"wind" + 0.001*"projects"
Topic: 2 Word: 0.001*"bank" + 0.001*"energy" + 0.001*"project" + 0.001*"power" + 0.001*"fund" + 0.001*"ps" + 0.001*"financial" + 0.001*"company" + 0.001*"wind" + 0.001*"development"
Topic: 3 Word: 0.001*"fund" + 0.001*"energy" + 0.001*"bank" + 0.001*"brac" + 0.001*"project" + 0.001*"." + 0.001*"investment" + 0.001*"access" + 0.001*"growth" + 0.001*"loan"
Topic: 4 Word: 0.001*"ps" + 0.001*":" + 0.001*"solar" + 0.001*"energy" + 0.001*"project" + 0.001*"fund" + 0.001*"impacts" + 0.001*"power" + 0.001*"-" + 0.001*"financial"
Topic: 5 Word: 0.001*"fund" + 0.001*"energy" + 0.001*"bank" + 0.001*"project" + 0.001*"usd" + 0.001*"microfinance" + 0.001*"coc

In [131]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(lda_model, bow_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.012230 -0.022476       1        1  41.331571
3      0.037588  0.013152       2        1  34.663621
1      0.000481  0.010770       3        1   9.888135
5     -0.007004 -0.020030       4        1   8.196441
0     -0.010208  0.010926       5        1   3.727399
2     -0.033088  0.007659       6        1   2.192834, topic_info=            Term          Freq         Total Category  logprob  loglift
154          the  10330.000000  10330.000000  Default  30.0000  30.0000
91            in   5457.000000   5457.000000  Default  29.0000  29.0000
16           and   7791.000000   7791.000000  Default  28.0000  28.0000
7              a   3745.000000   3745.000000  Default  27.0000  27.0000
115           of   5402.000000   5402.000000  Default  26.0000  26.0000
..           ...           ...           ...      ...      ...      ...
194  development     12.229540    670.063156   Topic6  -5.8379  -0.1835
162          usd      9.984380    384.184443   Topic6  -6.0408   0.1699
189      company     11.097661    588.796943   Topic6  -5.9350  -0.1514
118           on     12.281413    957.085566   Topic6  -5.8337  -0.5358
25            be     12.002210   1012.702628   Topic6  -5.8567  -0.6153

[493 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
1332       4  0.453658  "leapfrog
2547       5  0.520118     (fmo's
10043      4  0.452095  (globeleq
6397       2  0.310208       (mw)
6397       3  0.620416       (mw)
...      ...       ...        ...
5665       1  0.115228          •
5665       2  0.787393          •
5665       3  0.038409          •
5665       4  0.019205          •
5665       5  0.019205          •

[1008 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 2, 6, 1, 3])

In [132]:
pyLDAvis.gensim_models.prepare(lda_model_tfidf, tfidf_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.010115 -0.017275       1        1  21.944438
4     -0.013970  0.012879       2        1  19.829614
3      0.003377  0.001328       3        1  16.006951
1      0.002314  0.002904       4        1  14.671257
5      0.011014 -0.001986       5        1  14.306623
0      0.007380  0.002150       6        1  13.241118, topic_info=           Term      Freq     Total Category  logprob  loglift
2377       brac  1.000000  1.000000  Default  30.0000  30.0000
3791      cocoa  1.000000  1.000000  Default  29.0000  29.0000
287           :  2.000000  2.000000  Default  28.0000  28.0000
5692      dolma  0.000000  0.000000  Default  27.0000  27.0000
2218    locfund  0.000000  0.000000  Default  26.0000  26.0000
...         ...       ...       ...      ...      ...      ...
178         are  0.468345  3.996219   Topic6  -7.7179  -0.1221
15           an  0.443222  3.359633   Topic6  -7.7730  -0.0037
69    financing  0.443002  3.537332   Topic6  -7.7735  -0.0557
270        that  0.443685  3.724691   Topic6  -7.7720  -0.1058
102        loan  0.441492  3.709228   Topic6  -7.7769  -0.1066

[452 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
540       1  0.314647      %
540       2  0.314647      %
540       3  0.314647      %
544       1  0.380748      (
544       2  0.380748      (
...     ...       ...    ...
792       1  0.296029   wind
792       3  0.296029   wind
792       4  0.296029   wind
792       6  0.296029   wind
663       3  0.499871  women

[307 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 4, 2, 6, 1])